In [9]:
import enchant
import textblob
from textblob import TextBlob
from langdetect import detect
import collections
import re
from tqdm import tqdm

In [264]:
babyname_fn = "data/babynames.csv"
small_text_fn = 'data/responseText-0.txt'
text_fn = "data/responseText-500k.txt"

In [263]:
names = set()
with open(babyname_fn,"r") as f:
    for line in f:
        tokens = line.split(",")
        name = tokens[1][1:-1]
        names.add(name.lower())
print list(names)[:100]

['darryle', 'fawn', 'evalyn', 'sonji', 'darryll', 'sonja', 'elvina', 'suzann', 'timmothy', 'elva', 'woody', 'gabriella', 'gabrielle', 'cherrie', 'karlee', 'roena', 'lori', 'hermann', 'lora', 'karley', 'callie', 'dell', 'elihu', 'yulissa', 'errol', 'caryl', 'caryn', 'arminta', 'miller', 'eleanora', 'eleanore', 'rusty', 'francesca', 'melvin', 'lucious', 'joell', 'roddy', 'admiral', 'noreta', 'dorr', 'avery', 'herb', 'eunice', 'jasmine', 'camren', 'dora', 'averi', 'aleen', 'karis', 'lashawn', 'karim', 'karin', 'karie', 'zakary', 'golden', 'armstead', 'dorthy', 'lynne', 'dortha', 'siddie', 'doretta', 'keara', 'nicholaus', 'vanesa', 'owen', 'leesa', 'louetta', 'ardath', 'makenzie', 'janene', 'addyson', 'maximilian', 'burnett', 'matilde', 'matilda', 'melony', 'theodosia', 'julissa', 'charlton', 'terance', 'kraig', 'cielo', 'damarcus', 'dimple', 'margueritta', 'archibald', 'amiya', 'granville', 'tatsuo', 'ronnie', 'cadence', 'hoy', 'barbra', 'flossie', 'destiney', 'festus', 'alline', 'destine

In [3]:
def create_dictionary():
    extras = ["lol","rofl","omg","flickr","facebook","tumblr","instagram","myspace","lyft","airbnb","com","org","pdf",
              "edu","htm","html","eg","nb","sketchup","uber","gmail","http","www"] + list(names)
    d = enchant.Dict("en_US")
    for word in extras:
        d.add(word)
    return d

In [4]:
d = create_dictionary()

In [23]:
top_ngrams = collections.Counter()
def words(text): return re.findall("[a-z']+", text.lower()) 
with open(text_fn,"r") as f:
    for line in tqdm(f):
        try:
            lang = detect(line[:200])
        except Exception:
            continue
        if lang != "en":
            continue
        for word in words(line):
            if not d.check(word):
                top_ngrams[word.lower()] += 1

104446it [17:06, 96.51it/s]

KeyboardInterrupt: 

In [25]:
print len(top_ngrams)
hits = 0
for item,value in top_ngrams.iteritems():
    if value > 5:
        hits += 1
print hits

52150
5270


In [28]:
outfile = "top.txt"
with open(outfile,"w") as f:
    for item,value in top_ngrams.iteritems():
        if value > 5:
            f.write(item + "\n")

In [31]:
with open("curated.txt","r") as f:
    for line in tqdm(f):
        d.add(line[:-1])

In [75]:
import re, collections

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in tqdm(features):
        if d.check(f):
            model[f] += 1
    return model

NWORDS = train(words(file(small_text_fn).read()))

alphabet = list(c for c in "abcdefghijklmnopqrstuvwxyz'") + ["'t"]

def edits1(word):
    s = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in s if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in s if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in s for c in alphabet if b]
    inserts    = [a + c + b     for a, b in s for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

#def known_edits2(word):
#    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or [word] #or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [76]:
len(NWORDS)

45505

In [88]:
import codecs

num_correct = 0
num_incorrect = 0
oov = 0
incorrects = collections.Counter()
with codecs.open(text_fn,"r","utf-8") as f:
    for line in tqdm(f):
        try:
            lang = detect(line)
        except Exception:
            continue
        if lang != "en":
            continue
        for word in words(line.lower()):
            if word[-2:] == "'s":
                word = word[:-2]
            if not word:
                continue
            if word[0] == "'":
                word = word[1:]
            if not word:
                continue
            if word[-1] == "'":
                word = word[:-1]
            suggestions = [correct(word)]#d.suggest(word)
            if suggestions and suggestions[0].lower() == word:
                num_correct += 1
            elif suggestions:
                corrected = suggestions[0].lower() #correct(word)
                num_incorrect += 1
                incorrects[(word, corrected)] += 1
            else:
                oov += 1

In [89]:
print num_correct, num_incorrect, oov

51427385 259842 0


In [168]:
Edit = collections.namedtuple("Edit",["word","correct","edit"])
def get_edit(word, correct):
    if (word+"s") == correct:
        return None
    if word == (correct + "s"):
        return None
    s = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [Edit(word, a + b[1:], "delete {}".format(b[0])) for a, b in s if b]
    transposes = [Edit(word, a + b[1] + b[0] + b[2:], "transpose {}".format(b[0:2])) for a, b in s if len(b)>1]
    replaces   = [Edit(word, a + c + b[1:], "replace {}->{}".format(b[0],c)) for a, b in s for c in alphabet if b]
    inserts    = [Edit(word, a + c + b, "insert {}".format(c)) for a, b in s for c in alphabet]
    options = transposes + replaces + inserts + deletes
    for entry in options:
        if entry.correct == correct:
            return entry.edit
    return None

def edit_factory(word, correct):
    for index,(c,h) in enumerate(zip(word,correct)):
        if c != h:
            break
    else:
        index = len(word)
    if len(correct) > len(word):
        return lambda x,i : x[:i] + correct[index] + x[i:], index
    if len(correct) < len(word):
        return lambda x,i : x[:i] + x[i+1:], index
    if index+1 >= len(word) or word[index+1] == correct[index+1]:
        #mutation
        return lambda x,i : x[:i] + correct[index] + x[i+1:], index
    return lambda x,i : x[:i] + x[i+1] + x[i] + x[i+2:], index

def get_edit_function(word, correct):
    if (word+"s") == correct:
        return None
    if word == (correct + "s"):
        return None
    s = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [Edit(word, a + b[1:], "delete {}".format(b[0])) for a, b in s if b]
    transposes = [Edit(word, a + b[1] + b[0] + b[2:], "transpose {}".format(b[0:2])) for a, b in s if len(b)>1]
    replaces   = [Edit(word, a + c + b[1:], "replace {}->{}".format(b[0],c)) for a, b in s for c in alphabet if b]
    inserts    = [Edit(word, a + c + b, "insert {}".format(c)) for a, b in s for c in alphabet]
    options = transposes + replaces + inserts + deletes
    for entry in options:
        if entry.correct == correct:
            return entry.edit
    return None

In [92]:
hits = collections.Counter()
context = collections.defaultdict(list)
for (word,_),weight in tqdm(incorrects.iteritems()):
    target = correct(word)
    if word != target:
        edit = get_edit(word, target)
        if edit is not None:
            hits[edit] += weight
            context[edit].append((word, target))

In [290]:
#for entry in hits.most_common():
#    print entry, context[entry[0]][:10]
#    print "---"

In [97]:
transposes = collections.Counter()
replacements = collections.Counter()
for hit, weight in hits.iteritems():
    if "transpose" in hit:
        chars = hit[-2:]
        transposes[chars] += weight
    elif "replace" in hit:
        chars = hit[-4] + hit[-1]
        replacements[chars] += weight

In [94]:
#http://jayd.sauce.do/07/04/python-keyboard-heatmapper
from PIL import Image
key_location = {"Escape": (13, 10, 0), "F1": (78, 10, 0), "F2": (116, 10, 0), "F3": (154, 10, 0), "F4": (193, 10, 0), "F5": (253, 10, 0), "F6": (291, 10, 0), "F7": (329, 10, 0), "F8": (367, 10, 0), "F9": (428, 10, 0), "F10": (466, 10, 0), "F11": (504, 10, 0), "F12": (542, 10, 0), "Snapshot": (601, 10, 0), "Scroll`": (639, 10, 0), "Pause": (677, 10, 0), "Oem_3": (13, 82, 0), "1": (52, 83, 0), "2": (89, 82, 0), "3": (127, 82, 0), "4": (165, 82, 0), "5": (203, 82, 0), "6": (242, 82, 0), "7": (280, 82, 0), "8": (318, 82, 0), "9": (356, 82, 0), "0": (394, 82, 0), "Oem_Minus": (432, 82, 0), "Oem_Plus": (470, 82, 0), "Back": (508, 82, 1), "Insert": (603, 82, 0), "Home": (641, 82, 0), "Prior": (679, 82, 0), "NumLock": (738, 82, 0), "Divide-": (776, 82, 0), "Multiply*": (814, 82, 0), "Subtract": (852, 82, 0), "Tab": (13, 122, 2), "Q": (69, 122, 0), "W": (107, 122, 0), "E": (146, 122, 0), "R": (184, 122, 0), "T": (222, 122, 0), "Y": (260, 122, 0), "U": (297, 122, 0), "I": (336, 122, 0), "O": (374, 122, 0), "P": (412, 122, 0), "Oem_4": (449, 122, 0), "Oem_6": (487, 122, 0), "Oem_5": (526, 122, 3), "Delete": (602, 122, 0), "End": (640, 122, 0), "Next": (678, 122, 0), "Numpad7": (737, 122, 0), "Numpad8": (775, 122, 0), "Numpad9": (813, 122, 0), "Add": (852, 122, 4), "Capital": (13, 161, 5), "A": (79, 161, 0), "S": (117, 161, 0), "D": (156, 161, 0), "F": (194, 161, 0), "G": (232, 161, 0), "H": (270, 161, 0), "J": (308, 161, 0), "K": (346, 161, 0), "L": (384, 161, 0), "Oem_1": (422, 161, 0), "Oem_7": (461, 161, 0), "Return": (499, 161, 6), "Numpad4": (737, 161, 0), "Numpad5": (776, 161, 0), "Numpad6": (814, 161, 0), "Lshift": (13, 200, 7), "Z": (106, 200, 0), "X": (145, 200, 0), "C": (183, 200, 0), "V": (222, 200, 0), "B": (260, 200, 0), "N": (298, 200, 0), "M": (336, 200, 0), "Oem_Comma": (374, 200, 0), "Oem_Period": (413, 200, 0), "Oem_2": (451, 200, 0), "Rshift": (489, 200, 8), "Up": (641, 200, 0), "Numpad1": (738, 201, 0), "Numpad2": (775, 201, 0), "Numpad3": (814, 201, 0), "NumpadReturn": (851, 201, 9), "Lcontrol": (13, 240, 10), "Lwin": (69, 240, 11), "Lmenu": (119, 240, 11), "Space": (169, 240, 12), "Rmenu": (377, 240, 11), "Rwin": (427, 240, 11), "Apps": (476, 240, 11), "Rcontrol": (525, 240, 10), "Left": (603, 240, 0), "Down": (641, 240, 0), "Right": (679, 240, 0), "Numpad0": (738, 240, 13), "Decimal": (814, 240, 0)}
img_w = 31
img_h = 33
def build_heatmap(hitmap, target):
    biggest = float(max(v for k, v in hitmap.iteritems()))

    heatmap = Image.open("heat_gradient.png")
    im = Image.open("keyboard.png")

    for k, v in hitmap.iteritems():
        if(k not in key_location):
            continue

        heatbox = ((int((v/biggest)*500)-1), 0, int((v/biggest)*500)+1, 1)
        heatRegion = heatmap.crop(heatbox)

        heatColorInfo = heatRegion.getcolors()[0][1]

        newImg = Image.new('RGB', (img_w, img_h), heatColorInfo)

        box = (key_location[k][0], key_location[k][1], key_location[k][0]+img_w, key_location[k][1]+img_h)
        region = im.crop(box)
        region = Image.blend(region, newImg, .5)

        im.paste(region, box)
    
    newImg = Image.new('RGB', (img_w, img_h), (0,0,0))

    box = (key_location[target][0], key_location[target][1], key_location[target][0]+img_w, key_location[target][1]+img_h)
    region = im.crop(box)
    region = Image.blend(region, newImg, .5)

    im.paste(region, box)

    im.save('keyboard_heatmap.jpg')
    im.show()

In [98]:
def show_replacements_for(ch):
    for_char = collections.Counter()
    for chars,weight in replacements.iteritems():
        if chars[0] == ch:
            for_char[chars[1].upper()] += weight
            
    build_heatmap(for_char, ch.upper())
    
def show_transpositions_for(ch):
    for_char = collections.Counter()
    for chars,weight in transposes.iteritems():
        if ch in chars:
            for_char[chars[1].upper()] += weight
            for_char[chars[0].upper()] += weight
    print for_char
    build_heatmap(for_char, ch.upper())

In [265]:
show_replacements_for("p")

In [291]:
#print top_ngrams.most_common(500)

In [177]:
trigram_to_function = collections.defaultdict(list)
for (word,_),weight in tqdm(incorrects.iteritems()):
    target = correct(word)
    if word != target:
        word,target = target,word #get inverses
        edit, index = edit_factory(word, target)
        padded = "--"+word
        trigram = word[index-2:index+1]
        trigram_to_function[trigram].append(edit)
        

In [225]:
import numpy

def get_trigrams(word):
    padded = "--"+word
    trigrams = [padded[i:i+3] for i in xrange(len(padded)-2)]
    return trigrams

def inject_error_into_word(word, trigram_to_function):
    trigrams = [tri for tri in get_trigrams(word)]
    weights = numpy.array([len(trigram_to_function[tri]) for tri in trigrams])
    if sum(weights) == 0:
        return word
    weights = weights / float(weights.sum())
    index = numpy.random.choice(range(len(trigrams)), p=weights)
    trigram = trigrams[index]
    edit = numpy.random.choice(trigram_to_function[trigram])
    return edit(word, index)
    
def inject_errors(text, trigram_to_function, p):
    tokens = words(text)
    modified = []
    for word in tokens:
        if numpy.random.random() < p:
            modified.append(inject_error_into_word(word, trigram_to_function))
        else:
            modified.append(word)
    return " ".join(modified)

In [277]:
inject_error_into_word("food", trigram_to_function)

u'foood'

In [298]:
wikitext = 'Machine learning is closely related to computational statistics; a discipline that aims at the design of algorithms for implementing statistical methods on computers. It has strong ties to mathematical optimization, which delivers methods, theory and application domains to the field. Machine learning is employed in a range of computing tasks where designing and programming explicit algorithms is infeasible. Example applications include spam filtering, optical character recognition (OCR), search engines and computer vision. Machine learning is sometimes conflated with data mining, although that focuses more on exploratory data analysis. Machine learning and pattern recognition "can be viewed as two facets of the same field."'
inject_errors(wikitext, trigram_to_function, 0.05)

u'machine learning is closely related to computational statistics a discipline that aims at the design of algorithms for implementing statistical methods on computers it has strong ties to mathematical optimization which delivers methods theory and application domains to the field machine learning is employed in a range of computing tasks where designing and programming explicit algorithms is infeasible example applications include spam filtering optical character recognition ocr search engines and computer vision maxhine learning is somezimes conflated with data mining although that focuses more on exploreatory data analysis machine learning and pattern recognition cav be viewed as two facets of the same field'

In [293]:
import dill
with open("trigram_edits.pkl", "w") as f:
    dill.dump(trigram_to_function, f)